# OLS e REGRESSÃO ESPACIAL (IMÓVEIS URBANOS)

## Instalação de Pacotes Necessários

In [ ]:
%time
import os
import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns
import statsmodels.api as sm

!pip install --upgrade pandas
import pandas as pd
from pandas import read_csv

import matplotlib.pyplot as plt
%matplotlib inline

import math as math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

!pip install --upgrade scipy
import scipy as scipy
from scipy import stats
from scipy.stats import t

!pip install --upgrade shap
import shap as shap
shap.initjs()

In [ ]:
!pip install --upgrade geopandas
import geopandas

!pip install --upgrade pysal
import pysal
from pysal.model import spreg
from pysal.lib import weights
from pysal.lib import cg as geometry

!pip install --upgrade contextily
import contextily

import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

from spreg import OLS

from pointpats import centrography

## Importação de Dados Geolocalizados

In [ ]:
!pip install --upgrade gspread

In [ ]:
# autorização de acesso ao Google Drive 

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
planilha = gc.open('nome_basededados')
pagina = planilha.sheet1
pagina.row_values(1)

In [ ]:
df_Uniao = pd.DataFrame(pagina.get_all_records())

In [ ]:
df_Uniao.head(5)

In [ ]:
df_Uniao.columns

In [ ]:
df_Uniao.shape

In [ ]:
df_Uniao.isna().sum()

In [ ]:
df_Uniao = df_Uniao.dropna()

In [ ]:
df_Uniao = df_Uniao.reset_index(drop=True)

In [ ]:
df_Uniao.dtypes

In [ ]:
df_Uniao = df_Uniao.astype({"% Superior Completo": float})
df_Uniao = df_Uniao.astype({"% Rede Geral de Distribuicao de Agua": float})
df_Uniao = df_Uniao.astype({"% Microcomputador com Acesso a Internet": float})
df_Uniao = df_Uniao.astype({"Rendimento Domiciliar Medio": float})

df_Uniao.dtypes

In [ ]:
df_Uniao.describe()

In [ ]:
df_Uniao[df_Uniao["Fonte"]  == "ME"].count()

In [ ]:
df_Uniao[df_Uniao["Fonte"]  == "EB"].count()

In [ ]:
df_Uniao.drop([2730,4341,4581], axis=0, inplace=True)

# Abordagem 2

Transformação da área do terreno (área projetada horizontal do imóvel) e do valor total atualizado com logaritmo neperiano (sem remoção de *outliers*), logaritmo neperiano do valor total atualizado como variável dependente.

In [ ]:
df_Uniao.columns

In [ ]:
df_Uniao.describe()

In [ ]:
df_Uniao['CUB*AreaConstruida'] = df_Uniao['CUBJan22'] * df_Uniao['AreaConstruidaTratada']

In [ ]:
df_Uniao.columns

In [ ]:
var = 'ValorTotalAtualizado'

print(np.log(df_Uniao[var].min()))
print(np.log(df_Uniao[var].max()))

In [ ]:
# U1

col_list = df_Uniao.columns.tolist()

listaaremover = ['Fonte', 'Id', 'Município', 'UF', 'TipologiaMunicipalIBGE', 'IVS_2010', 'TaxaHomicidios_2019',
       'TipoImovel', 'CUBJan22', 'VidaUtil', 'AreaConstruidaTratada',
       'AreaProjetadaTratada', 'IndiceFipeZapAvaliacao', 'IndiceFipeZapJan22',
       'ValorTotalAtualizado', 'ValorUnitarioAtualizado', 'CodigoAP', 'NomeAP',
       '% Superior Completo', '% Rede Geral de Distribuicao de Agua',
       '% Microcomputador com Acesso a Internet', 'xcoord', 'ycoord', 'LatLong',
       'qtd_equipamento', 'qtd_hospital',
       'qtd_parque', 'qtd_delegacia', 'qtd_escola',
       'qtd_universidade', 'qtd_atracaoturistica', 'qtd_supermercado',
       'qtd_restaurante', 'qtd_padaria', 'qtd_cafeteria', 'qtd_loja', 'qtd_parada_onibus', 'qtd_estacao_vlt',
       'qtd_estacao_metro', 'qtd_estacao_trem', 'qtd_aeroporto',
       'acessibilidade', 'GoogleTrends2022', 'GrauUrbanizacaoIBGE', 'Rendimento Domiciliar Medio', 'qtd_shopping', 'qtd_estabelecimento', 'IdadeAparente']

for e in listaaremover:
  col_list.remove(e)

X_final_log_N1 = df_Uniao[col_list]

y_final_log_N1 = np.log(df_Uniao['ValorTotalAtualizado'])

y_final_log_N1 = y_final_log_N1[:,np.newaxis]

In [ ]:
# U2

col_list = df_Uniao.columns.tolist()

listaaremover = ['Fonte', 'Id', 'Município', 'UF', 'TipologiaMunicipalIBGE', 'IVS_2010', 'TaxaHomicidios_2019',
       'TipoImovel', 'CUBJan22', 'VidaUtil', 'AreaConstruidaTratada',
       'AreaProjetadaTratada', 'IndiceFipeZapAvaliacao', 'IndiceFipeZapJan22',
       'ValorTotalAtualizado', 'ValorUnitarioAtualizado', 'CodigoAP', 'NomeAP',
       '% Superior Completo', '% Rede Geral de Distribuicao de Agua',
       '% Microcomputador com Acesso a Internet', 'xcoord', 'ycoord', 'LatLong',
       'qtd_equipamento', 'qtd_hospital',
       'qtd_parque', 'qtd_delegacia', 'qtd_escola',
       'qtd_universidade', 'qtd_atracaoturistica', 'qtd_supermercado',
       'qtd_restaurante', 'qtd_padaria', 'qtd_cafeteria', 'qtd_loja', 'qtd_parada_onibus', 'qtd_estacao_vlt',
       'qtd_estacao_metro', 'qtd_estacao_trem', 'qtd_aeroporto',
       'acessibilidade', 'GoogleTrends2022', 'GrauUrbanizacaoIBGE', 'Rendimento Domiciliar Medio', 'qtd_shopping', 'qtd_estabelecimento']

for e in listaaremover:
  col_list.remove(e)

X_final_log_N2 = df_Uniao[col_list]

y_final_log_N2 = np.log(df_Uniao['ValorTotalAtualizado'])

y_final_log_N2 = y_final_log_N2[:,np.newaxis]

In [ ]:
# U3

col_list = df_Uniao.columns.tolist()

listaaremover = ['Fonte', 'Id', 'Município', 'UF', 'TipologiaMunicipalIBGE', 'IVS_2010', 'TaxaHomicidios_2019',
       'TipoImovel', 'CUBJan22', 'VidaUtil', 'AreaConstruidaTratada',
       'AreaProjetadaTratada', 'IndiceFipeZapAvaliacao', 'IndiceFipeZapJan22',
       'ValorTotalAtualizado', 'ValorUnitarioAtualizado', 'CodigoAP', 'NomeAP',
       '% Superior Completo', '% Rede Geral de Distribuicao de Agua',
       '% Microcomputador com Acesso a Internet', 'xcoord', 'ycoord', 'LatLong', 'qtd_hospital',
       'qtd_parque', 'qtd_delegacia', 'qtd_escola',
       'qtd_universidade', 'qtd_atracaoturistica', 'qtd_supermercado',
       'qtd_restaurante', 'qtd_padaria', 'qtd_cafeteria', 'qtd_loja', 'qtd_parada_onibus', 'qtd_estacao_vlt',
       'qtd_estacao_metro', 'qtd_estacao_trem', 'qtd_aeroporto',
       'acessibilidade', 'GoogleTrends2022', 'GrauUrbanizacaoIBGE']

for e in listaaremover:
  col_list.remove(e)

X_final_log_N3 = df_Uniao[col_list]

y_final_log_N3 = np.log(df_Uniao['ValorTotalAtualizado'])

y_final_log_N3 = y_final_log_N3[:,np.newaxis]

In [ ]:
X_final_log_N3.dtypes

## Testes de Normalidade

In [ ]:
# Teste de Shapiro-Wilk 

col_list = df_Uniao.columns.tolist()

listaaremover_testesnormalidade = ['Fonte', 'Id', 'Município', 'UF', 'TipologiaMunicipalIBGE', 'IVS_2010', 'TaxaHomicidios_2019',
       'TipoImovel', 'CUBJan22', 'VidaUtil', 'AreaConstruidaTratada',
       'AreaProjetadaTratada', 'IndiceFipeZapAvaliacao', 'IndiceFipeZapJan22',
       'ValorTotalAtualizado', 'ValorUnitarioAtualizado', 'CodigoAP', 'NomeAP',
       '% Superior Completo', '% Rede Geral de Distribuicao de Agua',
       '% Microcomputador com Acesso a Internet', 'xcoord', 'ycoord', 'LatLong', 'qtd_hospital',
       'qtd_parque', 'qtd_delegacia', 'qtd_escola',
       'qtd_universidade', 'qtd_atracaoturistica', 'qtd_supermercado',
       'qtd_restaurante', 'qtd_padaria', 'qtd_cafeteria', 'qtd_loja', 'qtd_parada_onibus', 'qtd_estacao_vlt',
       'qtd_estacao_metro', 'qtd_estacao_trem', 'qtd_aeroporto',
       'acessibilidade', 'GoogleTrends2022', 'GrauUrbanizacaoIBGE', 'Vocacao']

for e in listaaremover_testesnormalidade:
  col_list.remove(e)

X_testesnormalidade = df_Uniao[col_list]

logArea_normalidade = np.log(df_Uniao.loc[:,'AreaProjetadaTratada'])

X_testesnormalidade['logArea'] = logArea_normalidade

for variavel in X_testesnormalidade.columns:
  shapiro_test = stats.shapiro(X_testesnormalidade[variavel])
  print(variavel)
  print(shapiro_test)

In [ ]:
# Teste de Jarque-Bera

for variavel in X_testesnormalidade.columns:
  jarque_bera_test = stats.jarque_bera(X_testesnormalidade[variavel])
  print(variavel)
  print(jarque_bera_test)

Há evidências de que as variáveis têm assimetria e curtose significativamente
diferentes de uma distribuição normal. Portanto, não se aplicará a transformação StandardScaler().

## Partição em Conjuntos de Treinamento e de Teste

In [ ]:
logArea = np.log(df_Uniao.loc[:,'AreaProjetadaTratada'])

X_final_log_N1.loc[:,'logArea'] = logArea
X_final_log_N2.loc[:,'logArea'] = logArea
X_final_log_N3.loc[:,'logArea'] = logArea

In [ ]:
X_train_log_N1, X_test_log_N1, y_train_log_N1, y_test_log_N1 = train_test_split(X_final_log_N1, y_final_log_N1, test_size=0.33, shuffle=True, random_state=7, stratify=X_final_log_N1['Vocacao'])

In [ ]:
X_train_log_N2, X_test_log_N2, y_train_log_N2, y_test_log_N2 = train_test_split(X_final_log_N2, y_final_log_N2, test_size=0.33, shuffle=True, random_state=7, stratify=X_final_log_N2['Vocacao'])

In [ ]:
X_train_log_N3, X_test_log_N3, y_train_log_N3, y_test_log_N3 = train_test_split(X_final_log_N3, y_final_log_N3, test_size=0.33, shuffle=True, random_state=7, stratify=X_final_log_N3['Vocacao'])

In [ ]:
X_train_log_N1 = pd.get_dummies(X_train_log_N1, columns = ['Vocacao'])
X_test_log_N1 = pd.get_dummies(X_test_log_N1, columns = ['Vocacao'])

In [ ]:
X_train_log_N2 = pd.get_dummies(X_train_log_N2, columns = ['Vocacao'])
X_test_log_N2 = pd.get_dummies(X_test_log_N2, columns = ['Vocacao'])

In [ ]:
X_train_log_N3 = pd.get_dummies(X_train_log_N3, columns = ['Vocacao'])
X_test_log_N3 = pd.get_dummies(X_test_log_N3, columns = ['Vocacao'])

In [ ]:
X_train_log_N1.columns

In [ ]:
X_train_log_N2.columns

In [ ]:
X_train_log_N3.columns

In [ ]:
# U1

mms_N1_A = MinMaxScaler()

X_train_log_N1[['CapitalUF', 'IDHM_2010', 'Terreno', 'CUB*AreaConstruida',
       'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Misto',
       'Vocacao_Residencial']] = mms_N1_A.fit_transform(X_train_log_N1[['CapitalUF', 'IDHM_2010', 'Terreno', 'CUB*AreaConstruida',
       'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Misto',
       'Vocacao_Residencial']])

X_test_log_N1[['CapitalUF', 'IDHM_2010', 'Terreno', 'CUB*AreaConstruida',
       'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Misto',
       'Vocacao_Residencial']] = mms_N1_A.transform(X_test_log_N1[['CapitalUF', 'IDHM_2010', 'Terreno', 'CUB*AreaConstruida',
       'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Misto',
       'Vocacao_Residencial']])

In [ ]:
# U2

mms_N2_A = MinMaxScaler()

X_train_log_N2[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'CUB*AreaConstruida', 'Vocacao_Comercial',
       'Vocacao_Institucional', 'Vocacao_Misto', 'Vocacao_Residencial']] = mms_N2_A.fit_transform(X_train_log_N2[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'CUB*AreaConstruida', 'Vocacao_Comercial',
       'Vocacao_Institucional', 'Vocacao_Misto', 'Vocacao_Residencial']])

X_test_log_N2[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'CUB*AreaConstruida', 'Vocacao_Comercial',
       'Vocacao_Institucional', 'Vocacao_Misto', 'Vocacao_Residencial']] = mms_N2_A.transform(X_test_log_N2[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'CUB*AreaConstruida', 'Vocacao_Comercial',
       'Vocacao_Institucional', 'Vocacao_Misto', 'Vocacao_Residencial']])

In [ ]:
# U3

mms_N3_A = MinMaxScaler()

X_train_log_N3[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'Vocacao_Comercial', 'Vocacao_Institucional',
       'Vocacao_Misto', 'Vocacao_Residencial']] = mms_N3_A.fit_transform(X_train_log_N3[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'Vocacao_Comercial', 'Vocacao_Institucional',
       'Vocacao_Misto', 'Vocacao_Residencial']])

X_test_log_N3[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento_semshopping',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'Vocacao_Comercial', 'Vocacao_Institucional',
       'Vocacao_Misto', 'Vocacao_Residencial']] = mms_N3_A.transform(X_test_log_N3[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento_semshopping',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'Vocacao_Comercial', 'Vocacao_Institucional',
       'Vocacao_Misto', 'Vocacao_Residencial']])

## Modelagem Linear (OLS e Regressão Espacial)

In [ ]:
df_Uniao.info()

In [ ]:
col_list = df_Uniao.columns.tolist()

listaaremover = ['Fonte', 'Id', 'Município', 'UF', 'TipologiaMunicipalIBGE', 'IVS_2010', 'TaxaHomicidios_2019',
       'TipoImovel', 'CUBJan22', 'VidaUtil', 'AreaConstruidaTratada',
       'AreaProjetadaTratada', 'IndiceFipeZapAvaliacao', 'IndiceFipeZapJan22',
       'ValorTotalAtualizado', 'ValorUnitarioAtualizado', 'CodigoAP', 'NomeAP',
       '% Superior Completo', '% Rede Geral de Distribuicao de Agua',
       '% Microcomputador com Acesso a Internet', 'LatLong', 'qtd_hospital',
       'qtd_parque', 'qtd_delegacia', 'qtd_escola',
       'qtd_universidade', 'qtd_atracaoturistica', 'qtd_supermercado',
       'qtd_restaurante', 'qtd_padaria', 'qtd_cafeteria', 'qtd_loja', 'qtd_parada_onibus', 'qtd_estacao_vlt',
       'qtd_estacao_metro', 'qtd_estacao_trem', 'qtd_aeroporto',
       'acessibilidade', 'GoogleTrends2022', 'GrauUrbanizacaoIBGE']

for e in listaaremover:
  col_list.remove(e)

df_Uniao_Geo = df_Uniao[col_list]

logArea = np.log(df_Uniao['AreaProjetadaTratada'])
df_Uniao_Geo['logArea'] = logArea

LnValorTotalAtualizado = np.log(df_Uniao['ValorTotalAtualizado'])

In [ ]:
df_Uniao_Geo.columns

In [ ]:
joint_axes = sns.jointplot(
    x="xcoord", y="ycoord", data=df_Uniao_Geo, s=2.0, color = "green", xlim = [-80,-30], ylim = [-40,10], height = 20
)
contextily.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:4326",
    source=contextily.providers.CartoDB.PositronNoLabels,
);

In [ ]:
# Criação de Gráfico de Densidade de Imóveis Urbanos Anotados

f, ax = plt.subplots(1, figsize=(20, 20))

sns.kdeplot(
    df_Uniao_Geo["xcoord"],
    df_Uniao_Geo["ycoord"],
    n_levels=20,
    shade=True,
    alpha=0.55,
    cmap="viridis_r"
)

contextily.add_basemap(
    ax, source=contextily.providers.CartoDB.Positron,crs="EPSG:4326"
)

plt.xlim([-80,-30])
plt.ylim([-40,10])

plt.show()

In [ ]:
mean_center = centrography.mean_center(df_Uniao_Geo[["xcoord", "ycoord"]])
med_center = centrography.euclidean_median(df_Uniao_Geo[["xcoord", "ycoord"]])

In [ ]:
joint_axes = sns.jointplot(
    x="xcoord", y="ycoord", data=df_Uniao_Geo, s=0.75, height=9
)

joint_axes.ax_joint.scatter(
    *mean_center, color="red", marker="x", s=50, label="Mean Center"
)
joint_axes.ax_marg_x.axvline(mean_center[0], color="red")
joint_axes.ax_marg_y.axhline(mean_center[1], color="red")

joint_axes.ax_joint.scatter(
    *med_center,
    color="limegreen",
    marker="o",
    s=50,
    label="Median Center"
)
joint_axes.ax_marg_x.axvline(med_center[0], color="limegreen")
joint_axes.ax_marg_y.axhline(med_center[1], color="limegreen")

joint_axes.ax_joint.legend()

contextily.add_basemap(
    joint_axes.ax_joint, source=contextily.providers.CartoDB.Positron,crs="EPSG:4326"
)

joint_axes.ax_joint.set_axis_off()

plt.xlim([-80,-20])
plt.ylim([-40,10])

plt.show()

In [ ]:
df_Uniao_Geo["geometry"] = geopandas.points_from_xy(df_Uniao_Geo["xcoord"], df_Uniao_Geo["ycoord"])
df_Uniao_Geo = geopandas.GeoDataFrame(df_Uniao_Geo, crs="epsg:3857")

In [ ]:
radius = geometry.sphere.RADIUS_EARTH_KM
radius

In [ ]:
w = weights.DistanceBand.from_dataframe(df_Uniao_Geo, threshold=1.0, binary=False, radius = radius, alpha = -1.0)
w.transform = 'r'

In [ ]:
df_Uniao_Geo.columns

In [ ]:
df_Uniao_Geo = pd.get_dummies(df_Uniao_Geo, columns = ['Vocacao'])

In [ ]:
col_list = df_Uniao_Geo.columns.tolist()

listaaremover = ['xcoord','ycoord','geometry']

for e in listaaremover:
  col_list.remove(e)

df_Uniao_Geo = df_Uniao_Geo[col_list]

In [ ]:
df_Uniao_Geo.corr()

In [ ]:
plt.figure(figsize=(25, 25))
heatmap = sns.heatmap(df_Uniao_Geo.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Mapa de Calor de Correlações de Pearson', fontdict={'fontsize':18}, pad=12)

In [ ]:
df_Uniao_Geo.columns

In [ ]:
df_Uniao_Geo['LnValorTotalAtualizado'] = LnValorTotalAtualizado

In [ ]:
# U1
# explanatory_vars = ['CapitalUF', 'IDHM_2010', 'Terreno', 'CUB*AreaConstruida', 'logArea', 'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Residencial']

# U2
# explanatory_vars = ['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente', 'CUB*AreaConstruida', 'logArea', 'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Residencial']

# U3

explanatory_vars = ['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida', 'logArea', 'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Residencial']

In [ ]:
mms_geo = MinMaxScaler()

df_Uniao_Geo[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida',
       'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Misto',
       'Vocacao_Residencial']] = mms_geo.fit_transform(df_Uniao_Geo[['CapitalUF', 'IDHM_2010', 'Terreno', 'IdadeAparente',
       'Rendimento Domiciliar Medio', 'qtd_equipamento',
       'qtd_shopping', 'qtd_estabelecimento', 'CUB*AreaConstruida',
       'Vocacao_Comercial', 'Vocacao_Institucional', 'Vocacao_Misto',
       'Vocacao_Residencial']])

In [ ]:
df_Uniao_Geo.columns

In [ ]:
lnvalortotalatualizado = df_Uniao_Geo['LnValorTotalAtualizado']
y = np.array(lnvalortotalatualizado)
y.shape = (len(lnvalortotalatualizado), 1)

In [ ]:
X = []

for i in explanatory_vars:
  X.append(df_Uniao_Geo[i])

X = np.array(X).T

In [ ]:
print(X[1])

In [ ]:
len(y)

In [ ]:
len(X)

In [ ]:
variables = explanatory_vars

### Regressão Linear Múltipla (OLS)

In [ ]:
X_constante = sm.add_constant(X)
ols = OLS(y, X, name_y = 'LnValorTotalAtualizado', name_x = variables)

In [ ]:
print(ols.summary)

In [ ]:
print(ols.utu)

In [ ]:
MSE = ols.utu/len(X)

In [ ]:
RMSE = MSE ** (1/2)

In [ ]:
print(RMSE)

In [ ]:
X_constante = sm.add_constant(X)
model = sm.OLS(y, X_constante).fit()
predictions = model.predict(X_constante)

model.summary()

In [ ]:
# Cálculo das Distâncias de Cook

np.set_printoptions(suppress=True)

influence = model.get_influence()

cooks = influence.cooks_distance

print(cooks)

In [ ]:
plt.figure(figsize=(14, 8))
plt.scatter(df_Uniao_Geo.index, cooks[0])
plt.xlabel('x')
plt.ylabel('Distância de Cook')
plt.show()

In [ ]:
df_Cook = pd.DataFrame(cooks[0], columns = ['DistanciaCook'])

In [ ]:
df_Cook.shape

In [ ]:
df_Cook.dtypes

In [ ]:
df_Cook.query('DistanciaCook > 1.0')

### Regressão Espacial (S2SLS)

In [ ]:
lmgeo = spreg.GM_Lag(y, X, w=w, spat_diag=True, name_y = 'LnValorTotalAtualizado', name_x = variables)

In [ ]:
print(lmgeo.summary)

In [ ]:
spreg.r2(lmgeo)

In [ ]:
print(lmgeo.utu)

In [ ]:
MSE = lmgeo.utu/len(X)

In [ ]:
RMSE = MSE ** (1/2)

In [ ]:
print(RMSE)

I de Moran

In [ ]:
m = spreg.MoranRes(ols, w, z=True)

In [ ]:
print(round(m.I,4))

Significância I de Moran

In [ ]:
print(m.p_norm)

Reprodução dos experimentos utilizando os conjuntos particionados em treinamento e teste.